In [13]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image

In [14]:
OVERPASS_URL = 'http://overpass-api.de/api/interpreter'

To get all nodes that lay on a way in a radius of 200 meters around a point

In [37]:
lat = 50.85866
lon = 3.29640

In [ ]:
def get_intersection_coordinates(lat, lon) : 
    # some lists and dataframes to store the data
    nodes = []
    intersection_nodes = []
    df_nodes = pd.DataFrame(columns=['id', 'lat', 'lon'])
    df_intersections = pd.DataFrame(columns=['lat', 'lon'])


    # first get the ways around the point 
    overpass_query = f"""[out:json];
                        way(around: 100, {lat},{lon})->.w;
                        node(around: 100, {lat},{lon})->.n;
                        node.w.n;
                        out;"""
    
    overpass_url = "http://overpass-api.de/api/interpreter"

    response = requests.get(overpass_url, params={'data': overpass_query})

    print(response.json())
    for item in response.json()['elements']:
        # print(item)
        if item['type'] == 'node':
            df_nodes.loc[len(df_nodes)] = {'id': item['id'], 'lat': item['lat'], 'lon': item['lon']}
        
    for item in response.json()['elements']: 
        if item['type'] == 'way':
            for node_id in item['nodes']:
                if node_id not in nodes:
                    nodes.append(node_id)
                else:
                    # only if node already in nodes, it is an intersection
                    # now check if the intersection is already saved in intersection_nodes
                    if node_id not in intersection_nodes:
                        intersection_nodes.append(node_id)
                        
                        # Find the row in df_nodes where 'id' matches the current node
                        matching_row = df_nodes[df_nodes['id'] == node_id]
                        
                        # Check if a matching row was found
                        if not matching_row.empty:
                            latitude = matching_row['lat'].values[0]
                            longitude = matching_row['lon'].values[0]
                            df_intersections.loc[len(df_intersections)] = {'lat': latitude, 'lon': longitude}
    
    print(f"Length of df_intersections: {len(df_intersections)}")
    
    return df_intersections

In [58]:
def get_intersection_coordinates(lat, lon) : 
    # some lists and dataframes to store the data
    nodes = []
    intersection_nodes = []
    df_nodes = pd.DataFrame(columns=['id', 'lat', 'lon'])
    df_intersections = pd.DataFrame(columns=['lat', 'lon'])


    # first get the ways around the point 
    # .w is a temporary variable that stores the result of the first query
    # node(way_cnt:2-);   // select those nodes that are members of two or more of the given ways
    overpass_query = f"""[out:json];
                        way(around: 100, {lat},{lon})->.w;
                        node(way_cnt.w:2-);  
                        out;"""

    response = requests.get(OVERPASS_URL, params={'data': overpass_query})

    for item in response.json()['elements']:
        df_intersections.loc[len(df_intersections)] = {'lat': item['lat'], 'lon': item['lon']}


    print(df_intersections)
    return df_intersections

In [59]:
get_intersection_coordinates(lat, lon)

          lat       lon
0   50.859016  3.297076
1   50.859517  3.296069
2   50.858547  3.296203
3   50.858904  3.297153
4   50.858669  3.297557
5   50.858737  3.297643
6   50.858755  3.297705
7   50.858757  3.297766
8   50.858741  3.297840
9   50.858550  3.298432
10  50.858673  3.297837
11  50.858679  3.297788
12  50.858673  3.297726
13  50.858657  3.297677
14  50.858634  3.297634
15  50.858585  3.297557
16  50.858890  3.296966
17  50.858632  3.297953
18  50.859409  3.298077
19  50.859672  3.296283
20  50.859204  3.296691
21  50.859021  3.297067
22  50.858948  3.296634
23  50.859018  3.296542
24  50.858896  3.296539
25  50.858967  3.296441
26  50.859228  3.296359
27  50.859153  3.296269
28  50.859211  3.296180
29  50.859276  3.296258
30  50.859248  3.296060
31  50.859262  3.296076
32  50.859326  3.296153
33  50.859309  3.295976


,lat,lon
0,50.859016,3.297076
1,50.859517,3.296069
2,50.858547,3.296203
3,50.858904,3.297153
4,50.858669,3.297557
5,50.858737,3.297643
6,50.858755,3.297705
7,50.858757,3.297766
8,50.858741,3.297840
9,50.858550,3.298432
